In [7]:
!pip install tensorflow pandas numpy

In [8]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

In [9]:
WINDOW_PAST = 10
WINDOW_FUTURE = 10

In [10]:
df = pd.read_csv('data.csv')
df = df.dropna()

print(df.head(5))

             timestamp  temparature  humidity  pressure
0  2025-12-14 20:35:03       34.937      36.0  1000.036
1  2025-12-14 20:35:08       34.899      35.8  1000.023
2  2025-12-14 20:35:13       34.899      36.1  1000.039
3  2025-12-14 20:35:18       34.955      36.0  1000.028
4  2025-12-14 20:35:23       35.049      36.0  1000.055


In [11]:
X,y = [], []

vals = df[['temparature', 'humidity']].values.astype(np.float32)

window_groups = len(vals) - WINDOW_PAST - WINDOW_FUTURE + 1

for i in range(window_groups):
    x_past = vals[i:i+WINDOW_PAST]
    future_window = vals[i+WINDOW_PAST : i+WINDOW_PAST+WINDOW_FUTURE]
    y_future_mean = future_window.mean(axis=0)

    X.append(x_past)
    y.append(y_future_mean)

X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.float32)

print(X.shape, y.shape)

(475, 10, 2) (475, 2)


In [12]:
model = models.Sequential([
    layers.Input(shape=(WINDOW_PAST, 2)),
    layers.Flatten(),
    layers.Dense(32, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(2)
])

model.compile(
    optimizer="adam",
    loss="mse"
)

model.fit(X, y, epochs=40, batch_size=32, validation_split=0.2)

Epoch 1/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 1614.7698 - val_loss: 1320.6211
Epoch 2/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1204.6681 - val_loss: 1108.0212
Epoch 3/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 991.4219 - val_loss: 799.5237
Epoch 4/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 706.8411 - val_loss: 501.5276
Epoch 5/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 446.3465 - val_loss: 286.9150
Epoch 6/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 274.0187 - val_loss: 176.1439
Epoch 7/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 180.2128 - val_loss: 106.8905
Epoch 8/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 112.9117 - val_loss: 52.4542
Epoch 9/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 58.8604 - val_loss: 19.0521
Epoch 10/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 24.4914 - val_loss: 3.6176
Epoch 11/40
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 7.8174 - val_loss: 0.0229
Epoch 12/40
12/12 ━━━━━━━━━━━━━━━━━

In [13]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

with open("comfort_forecast.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model saved")

Saved artifact at '/tmp/tmp6essllnq'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 10, 2), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  139265089651920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139265089652880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139265090961040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139265089652688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139265089651728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139265089650960: TensorSpec(shape=(), dtype=tf.resource, name=None)
TFLite model saved


In [14]:
from google.colab import files
files.download("comfort_forecast.tflite")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>